# Ambiente

In [ ]:
!pip install transformers

# Tokenizadores



Os tokenizadores são independentes do framework mas precisam ser os mesmos usados no modelo.
Nós especificamos o tipo de tensores que devem sair (`return_tensors='pt'` ou `return_tensors='tf'`).


Estes tokenizadores possuem o vocabulário construido durante o pré-treinamento e também a técnica de tokenização a ser utilizada.

In [ ]:
from transformers import AutoTokenizer

In [ ]:
#carregando 4 tokenizadores para ver as diferenças
tokenizer_DBERT = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokenizer_BERT = AutoTokenizer.from_pretrained("bert-base-uncased")
tokenizer_MBERT = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased") #104 linguas
tokenizer_BERTIMBAU = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
tokenizer_GPT2 = AutoTokenizer.from_pretrained("gpt2") # é o gpt2 small

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Nos tokenizadores também estão especificados os tokens especiais usados pelo modelo de linguagem.

In [ ]:
print(tokenizer_DBERT.all_special_tokens, '=>', tokenizer_DBERT.all_special_ids)
print(tokenizer_BERT.all_special_tokens, '=>', tokenizer_BERT.all_special_ids)
print(tokenizer_MBERT.all_special_tokens, '=>', tokenizer_MBERT.all_special_ids)
print(tokenizer_BERTIMBAU.all_special_tokens, '=>', tokenizer_BERTIMBAU.all_special_ids)
print(tokenizer_GPT2.all_special_tokens, '=>', tokenizer_GPT2.all_special_ids)

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]'] => [100, 102, 0, 101, 103]
['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]'] => [100, 102, 0, 101, 103]
['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]'] => [100, 102, 0, 101, 103]
['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]'] => [100, 102, 0, 101, 103]
['<|endoftext|>'] => [50256]


O tamanho dos vocabulários é definido no pré-treinamento, pode ser um número  definido de alguma foram por quem criou o modelo. Normalmente conterá os tokens mais frequentes.

In [ ]:
#imprimindo os tamanhos do vocabulário de cada tokenizador
tokenizer_DBERT.vocab_size, tokenizer_BERT.vocab_size, tokenizer_MBERT.vocab_size, tokenizer_BERTIMBAU.vocab_size, tokenizer_GPT2.vocab_size

(30522, 30522, 105879, 29794, 50257)

BERT utiliza o tokenizador [WordPiece](https://huggingface.co/course/chapter6/6?fw=pt).

O WordPiece começa com um pequeno vocabulário incluindo os tokens especiais usados pelo modelo e o alfabeto inicial (caracteres únicos). Ele identifica subpalavras adicionando um prefixo (por exemplo ##, no caso do BERT). Cada palavra é inicialmente dividida adicionando esse prefixo a todos os caracteres dentro da palavra. Assim, por exemplo, a palavra "word" é inicialmente dividida em:

`w ##o ##r ##d`

Então, o WordPiece aprende as regras de unificação desses tokens, calculando um score para cada par, usando a seguinte fórmula:

`score=(freq_of_pair)/(freq_of_first_element×freq_of_second_element)
`

Ao dividir a frequência do par pelo produto das frequências de cada uma de suas partes, o algoritmo prioriza a junção dos pares onde as partes individuais são menos frequentes no vocabulário. Por exemplo, não irá necessariamente fundir ("un", "##able") mesmo que esse par ocorra com muita frequência no vocabulário, porque os dois pares "un" e "##able" provavelmente aparecerão em um muitas outras palavras e têm uma alta frequência. Por outro lado, um par como ("hu", "##gging") provavelmente será mesclado mais rapidamente (supondo que a palavra "hugging" apareça com frequência no vocabulário), pois "hu" e "##gging" provavelmente serão menos frequentes individualmente.

In [ ]:
#imprime as primeiras 15 palavras do vocabulário
print(list(tokenizer_BERT.vocab)[0:15])

['prints', '##ske', 'reigned', 'collarbone', 'geelong', 'failure', '##kk', 'corsica', 'slept', 'panel', '##tier', 'heroic', 'anson', 'melted', '610']


In [ ]:
#tokenizando algumas palavras de exemplo com o tokenizador do BERT
ids = tokenizer_BERT('Superbizarre or superb?')['input_ids']
print(ids)
print(tokenizer_BERT.batch_decode(ids))

[101, 21688, 21335, 14343, 2030, 21688, 1029, 102]
['[CLS]', 'superb', '##iza', '##rre', 'or', 'superb', '?', '[SEP]']


O GPT2 utiliza a tokenização [Byte Pair Encoding (BPE)](https://huggingface.co/course/chapter6/5?fw=pt), iniciando o processo com o alfabeto e em símbolos unitários no nivel de byte.

No vocabulário do GPT2 a ocorrência `Ġ` se refere ao espaço na frente da palavra, ao decodificar esse caractere desaparece.

In [ ]:
print(tokenizer_GPT2.vocab)

{'Ġmayoral': 43811, 'Ġcooled': 32162, 'Ġ__': 11593, 'rational': 20310, 'Ġcoh': 16883, 'onomous': 38175, 'å¤§': 32014, '</': 3556, 'ĠNHS': 18183, 'Progress': 32577, 'ĠBett': 43599, 'ę': 213, 'ite': 578, 'ĠFlynn': 14654, 'ĠBangl': 17891, 'ĠAdd': 3060, 'ĠNing': 37400, 'Ġrecycle': 48914, 'ĠSongs': 31772, 'Ġdissect': 38319, 'ĠFEMA': 43575, 'Ġendeavors': 40273, 'Ġamy': 38544, 'Ġdinners': 42541, 'Ġcavity': 31643, 'Ġprope': 44408, 'ĠTreat': 11217, 'Ġframework': 9355, 'Ġdemonstrated': 9555, 'ĠGow': 41396, 'Guest': 42481, 'ĠGeForce': 12975, 'Ġenvironments': 12493, 'Construct': 42316, 'Ġmolecule': 27756, 'Ġdebit': 30977, 'âĢĶ-': 44839, 'Ġroster': 9354, 'Package': 27813, 'ĠGa': 12822, 'technology': 45503, 'Initialized': 28500, 'ĠACE': 40488, 'ownt': 6887, 'ĠMile': 26004, 'ĠJud': 4794, 'Ġcausing': 6666, 'plus': 9541, 'Ġmobility': 15873, 'Playing': 36530, 'tmp': 22065, ')"': 16725, 'ĠEnder': 45322, 'ĠLucia': 48158, '1920': 40454, 'rait': 12907, 'Ġline': 1627, 'features': 40890, 'ĠYs': 43740, 'ĠCorta

In [ ]:
id = tokenizer_GPT2.vocab['Ġadvocacy']
print(id)
tokenizer_GPT2.decode(id)

15364


' advocacy'

In [ ]:
#tokenizando algumas palavras de exemplo com o tokenizador do GPT
ids = tokenizer_GPT2('The book is on the tabel')['input_ids']
print(ids)
print(tokenizer_GPT2.batch_decode(ids))

[464, 1492, 318, 319, 262, 34754, 254]
['The', ' book', ' is', ' on', ' the', ' �', '�']


In [ ]:
#tokenizando algumas palavras de exemplo com o tokenizador do BERT
cod = tokenizer_BERT('super bizarre! superbizarre abnegation!!' )
print(tokenizer_BERT.batch_decode(cod['input_ids']))
print(cod['input_ids'])

['[CLS]', 'super', 'bizarre', '!', 'superb', '##iza', '##rre', 'ab', '##ne', '##gation', '!', '!', '[SEP]']
[101, 3565, 13576, 999, 21688, 21335, 14343, 11113, 2638, 12540, 999, 999, 102]


In [ ]:
#tokenizando algumas palavras de exemplo com o tokenizador do BERT
cod = tokenizer_BERTIMBAU('O livro está sobre a paralelepípedo!!' )
print(tokenizer_BERTIMBAU.batch_decode(cod['input_ids']))
print(cod['input_ids'])

['[CLS]', 'O', 'livro', 'está', 'sobre', 'a', 'paral', '##ele', '##p', '##íp', '##ed', '##o', '!', '!', '[SEP]']
[101, 231, 1722, 698, 498, 123, 4405, 583, 22291, 831, 430, 22280, 106, 106, 102]


In [ ]:
#tokenizando algumas palavras de exemplo com o tokenizador do GPT
cod = tokenizer_GPT2('super bizarre! superbizarre abnegation!!' )
print(tokenizer_GPT2.batch_decode(cod['input_ids']))
print(cod['input_ids'])

['super', ' bizarre', '!', ' superb', 'izarre', ' ab', 'neg', 'ation', '!!']
[16668, 13699, 0, 21840, 12474, 450, 12480, 341, 3228]


A codificação gera os `input_ids`, `attention_masks` e `token_type_ids` (este último no caso do BERT) que são usados para o treinamento do modelo.

O padding neste exemplo utiliza o comprimento da maior sequência (i.e., são inseridos caracteres de padding na menor sequência).

In [ ]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer_BERT(raw_inputs, padding=True, truncation=True, return_tensors="pt")
for k in inputs.keys():
    print(f'{k}: {inputs[k]}')


input_ids: tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]])
token_type_ids: tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])


<a id='pytorch_native'></a>

<a id='additional-resources'></a>